In [ ]:
from functools import partial
import sklearn.metrics
import networkx as nx
from glob import glob
import igraph as ig
import pandas as pd
import numpy as np
import re
import os

In [ ]:
# Make dirs for plots
for algorithm in ['Leiden', 'Multi-level', 'Fast-greedy']:
    if not os.path.exists(algorithm):
        os.makedirs(algorithm)

In [ ]:
networks = []
leiden_mod = []
multilevel_mod = []
fastgreedy_mod = [] 

for path in glob("../real/*.net"):
    name = os.path.basename(path)[:-4]
    networks.append(name)
    g = ig.load(path).simplify()
    layout = g.layout_auto()
    #plot = partial(ig.plot, layout=layout, bbox=(300, 300), margin=20, mark_groups=True)
    plot = partial(ig.plot, layout=layout)

    #LEIDEN
    leiden = g.community_leiden('modularity', n_iterations=-1)
    leiden_mod.append(leiden.modularity)
    plot(leiden, f"Leiden/{name}.png")
    
    #MULTI-LEVEL
    multilevel = g.community_multilevel()
    multilevel_mod.append(multilevel.modularity)
    plot(multilevel, f"Multi-level/{name}.png")
    
    #FAST GREEDY
    fastgreedy = g.community_fastgreedy().as_clustering()
    fastgreedy_mod.append(fastgreedy.modularity)
    plot(fastgreedy, f"Fast-greedy/{name}.png")
    
    #if ground truth exists - compare
    reference_path = path.replace(".net", ".clu")
    if os.path.exists(reference_path):
        with open(reference_path, 'r') as f:
            next(f)
            reference = [int(line)-1 for line in f]
        #print(compare(leiden, reference, method = c("vi", "nmi","split.join", "rand","adjusted.rand")))

#SAVE TO TABLE
mod = {
    'Leiden': leiden_mod,
    'Multi-level': multilevel_mod,
    'Fast-greedy': fastgreedy_mod
}
df = pd.DataFrame(mod, index=networks)
print("\nMODULARITY TABLE\n", df)